<a href="https://colab.research.google.com/github/DavoodSZ1993/Pytorch_Basics/blob/main/pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Some Useful Notes on PyTorch Module

### `torchvision.transforms` Module: 

Transforms are common image transformations. 

* `transforms.Compose()`: Composes several transforms together.
* `transforms.Resize()`: Resizes the input image to the given size.
* `transforms.ToTensor()`: Converts a `PIL Image ` or `numpy.ndarray` (H x W x C) to tensor (C x H x W).

### `torchvision.datasets` Module:

* `datasets.FashionMNIST()`: Fashion-MNIST dataset, consisting of a training set of 60,000 examples and a test set of 10,000 examples. The dataset contains 28x28 grayscale images associated with 10 classes (labels).

In [4]:
from torchvision import transforms, datasets
from IPython import display

train_data = datasets.FashionMNIST(root='../data', train=True, download=True)
val_data = datasets.FashionMNIST(root='../data', train=False, download=True)
display.clear_output()

len(train_data), len(val_data)

(60000, 10000)